# Predicting daily stock market movement with machine learning
Week 2 - Obtain and prepare data
- Historical stock price data will be obtained from the Yahoo Finance API, via yfinance
- Documentation links for yfinance:

> https://pypi.org/project/yfinance/

> https://github.com/ranaroussi/yfinance

In [1]:
# import libraries
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

## Obtaining data
- Tesla stock will be used for this project

In [2]:
day_df = yf.download("TSLA",           # obtaining data for Tesla stock       
                     period='1y',      # 1 year of data beginning current day, previous year
                     interval='1d')    # daily statistics

hr_df = yf.download("TSLA", 
                    period='1y', 
                    interval='1h',     # hour interval 
                    prepost=True)      # inlcude pre-market & post-market data

min_df = yf.download("TSLA", 
                    period='60d',      # max period for this interval is 60 days
                    interval='2m',     # 2 minute intervals 
                    prepost=True)      

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## Inspecting data

In [3]:
# Daily data (1 yr)

day_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-03-18 to 2023-03-17
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       251 non-null    float64
 1   High       251 non-null    float64
 2   Low        251 non-null    float64
 3   Close      251 non-null    float64
 4   Adj Close  251 non-null    float64
 5   Volume     251 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.7 KB


- No missing values to deal with

In [4]:
day_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-18,291.496674,302.616669,289.130005,301.796661,301.796661,100414200
2022-03-21,304.993347,314.283325,302.363342,307.053345,307.053345,81981600
2022-03-22,310.000000,332.619995,307.250000,331.326660,331.326660,105868500
2022-03-23,326.646667,346.899994,325.466675,333.036682,333.036682,120676200
2022-03-24,336.576660,341.496674,329.600006,337.973328,337.973328,68920800


In [5]:
day_df.tail()

# final row reflects current day data, although there is a delay

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-03-13,167.460007,177.350006,163.910004,174.479996,174.479996,167790300
2023-03-14,177.309998,183.800003,177.139999,183.259995,183.259995,143717900
2023-03-15,180.800003,182.339996,176.029999,180.449997,180.449997,145995600
2023-03-16,180.369995,185.809998,178.839996,184.130005,184.130005,121136800
2023-03-17,184.520004,186.220001,177.330002,180.130005,180.130005,132936600


- The '1d' interval has returned the following features:
    - date (index)
    - opening price
    - daily high
    - daily low
    - closing price
    - adjusted closing price
    - volumne of shares traded

In [6]:
# Hourly data (1y)

hr_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4179 entries, 2022-03-18 04:00:00-04:00 to 2023-03-17 19:00:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4179 non-null   float64
 1   High       4179 non-null   float64
 2   Low        4179 non-null   float64
 3   Close      4179 non-null   float64
 4   Adj Close  4179 non-null   float64
 5   Volume     4179 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 228.5 KB


- No missing values to deal with

In [7]:
hr_df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-03-18 04:00:00-04:00,288.58334,289.99667,288.00000,289.53333,289.53333,0
2022-03-18 05:00:00-04:00,289.46667,289.83334,288.80667,289.21000,289.21000,0
2022-03-18 06:00:00-04:00,289.12332,289.66666,288.21000,288.44670,288.44670,0
2022-03-18 07:00:00-04:00,288.40000,288.83334,287.50333,288.03333,288.03333,0
2022-03-18 08:00:00-04:00,288.15000,291.25967,287.00000,288.65665,288.65665,0


In [8]:
hr_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-03-17 15:30:00-04:00,180.070007,180.449997,179.429993,180.110001,180.110001,9480575
2023-03-17 16:00:00-04:00,180.100000,180.416000,179.930000,179.980000,179.980000,0
2023-03-17 17:00:00-04:00,184.540000,256.118400,133.415910,179.650000,179.650000,0
2023-03-17 18:00:00-04:00,179.730000,179.800000,179.210000,179.365000,179.365000,0
2023-03-17 19:00:00-04:00,179.260000,180.130000,178.720000,178.930000,178.930000,0


- The same columns have been returned but now the time is included in the index and volumne is only present during intraday trading hours

In [9]:
# 2-minute interval data (60 days)

min_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14220 entries, 2023-02-02 19:00:00-05:00 to 2023-03-17 19:58:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       14220 non-null  float64
 1   High       14220 non-null  float64
 2   Low        14220 non-null  float64
 3   Close      14220 non-null  float64
 4   Adj Close  14220 non-null  float64
 5   Volume     14220 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 777.7 KB


- No missing values to deal with

In [10]:
min_df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-02-02 19:00:00-05:00,184.700,184.780,184.65,184.78,184.78,0
2023-02-02 19:02:00-05:00,184.715,184.780,184.65,184.74,184.74,0
2023-02-02 19:04:00-05:00,184.750,184.890,184.71,184.78,184.78,0
2023-02-02 19:06:00-05:00,184.800,184.915,184.75,184.82,184.82,0
2023-02-02 19:08:00-05:00,184.840,184.900,184.79,184.81,184.81,0


In [11]:
min_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-03-17 19:50:00-04:00,179.1050,179.27,179.10,179.15,179.15,0
2023-03-17 19:52:00-04:00,179.1800,179.25,179.09,179.13,179.13,0
2023-03-17 19:54:00-04:00,179.1100,179.15,179.03,179.10,179.10,0
2023-03-17 19:56:00-04:00,179.1000,179.10,179.00,179.05,179.05,0
2023-03-17 19:58:00-04:00,179.0101,179.07,178.72,178.93,178.93,0


## Preparing data

With no missing values to deal with, I'll move straight into feature engineering:
- The focus will be on the adjusted closing price (Adj Close) and % change between intervals
- Absolute percent change will be calcuated for EDA & visualization, while natural log returns will be calculated for analysis & machine learning purposes
- UDFs will be created to avoid redundancy

In [12]:
def add_returns(df):
    '''
    Adds column with adjusted close percentage change between intervals;
    also adds column with the natural log of percentage change
    '''
    
    df['% change'] = df['Adj Close'] / df['Adj Close'].shift(1) - 1
    df['% change (ln)'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))                  
    
    # Division by shifted values creates useless NA values
    df.dropna(inplace=True) 

In [13]:
# apply to each dataframe

df_list = [day_df, hr_df, min_df]

for df in df_list:
    add_returns(df)

In [14]:
def add_sessions(df):
    '''
    Creates new date & time columns for readability;
    new column that indicates whether trade occurred during intraday period;
    new column that indicates whether the period is before the market opens or after it closesas well as if the
    '''

    df['Date'] = pd.to_datetime(df.index)
    df['time'] = df['Date'].dt.strftime ('%H:%M')
    df['date'] = df['Date'].dt.strftime ('%Y-%m-%d')
    df.drop(columns='Date', axis=1, inplace=True)
    
    df['intraday'] = ''
    df['after/before'] = ''
    
    for i in range(len(df)):

        if df.time[i] > '15:30':        
            df.intraday[i] = 'no'
            df['after/before'][i] = 'after close'

        elif df.time[i] < '09:30':
            df.intraday[i] = 'no'
            df['after/before'][i] = 'before open'

        else:
            df.intraday[i] = 'yes'
            df['after/before'][i] = 'intraday'

In [15]:
# no sessions for day_df

df_list = [hr_df, min_df]

for df in df_list:
    add_sessions(df)

## Final data check

In [16]:
day_df.head()

,Open,High,Low,Close,Adj Close,Volume,% change,% change (ln)
Date,,,,,,,,
2022-03-21,304.993347,314.283325,302.363342,307.053345,307.053345,81981600,0.017418,0.017268
2022-03-22,310.000000,332.619995,307.250000,331.326660,331.326660,105868500,0.079052,0.076083
2022-03-23,326.646667,346.899994,325.466675,333.036682,333.036682,120676200,0.005161,0.005148
2022-03-24,336.576660,341.496674,329.600006,337.973328,337.973328,68920800,0.014823,0.014714
2022-03-25,336.000000,340.600006,332.440002,336.880005,336.880005,62031600,-0.003235,-0.003240


In [17]:
hr_df.head()

,Open,High,Low,Close,Adj Close,Volume,% change,% change (ln),time,date,intraday,after/before
Datetime,,,,,,,,,,,,
2022-03-18 05:00:00-04:00,289.46667,289.83334,288.80667,289.21000,289.21000,0,-0.001117,-0.001117,05:00,2022-03-18,no,before open
2022-03-18 06:00:00-04:00,289.12332,289.66666,288.21000,288.44670,288.44670,0,-0.002639,-0.002643,06:00,2022-03-18,no,before open
2022-03-18 07:00:00-04:00,288.40000,288.83334,287.50333,288.03333,288.03333,0,-0.001433,-0.001434,07:00,2022-03-18,no,before open
2022-03-18 08:00:00-04:00,288.15000,291.25967,287.00000,288.65665,288.65665,0,0.002164,0.002162,08:00,2022-03-18,no,before open
2022-03-18 09:00:00-04:00,288.57666,289.19998,287.66666,288.26334,288.26334,0,-0.001363,-0.001363,09:00,2022-03-18,no,before open


In [18]:
min_df.head()

,Open,High,Low,Close,Adj Close,Volume,% change,% change (ln),time,date,intraday,after/before
Datetime,,,,,,,,,,,,
2023-02-02 19:02:00-05:00,184.715,184.780,184.65,184.74,184.74,0,-0.000216,-0.000216,19:02,2023-02-02,no,after close
2023-02-02 19:04:00-05:00,184.750,184.890,184.71,184.78,184.78,0,0.000217,0.000216,19:04,2023-02-02,no,after close
2023-02-02 19:06:00-05:00,184.800,184.915,184.75,184.82,184.82,0,0.000216,0.000216,19:06,2023-02-02,no,after close
2023-02-02 19:08:00-05:00,184.840,184.900,184.79,184.81,184.81,0,-0.000054,-0.000054,19:08,2023-02-02,no,after close
2023-02-02 19:10:00-05:00,184.830,185.000,184.81,185.00,185.00,0,0.001028,0.001028,19:10,2023-02-02,no,after close


## Week 2 summary
For week 2, the data collection and preparation stage, I obtained historical data for Tesla stock from yfinance. Data was collected for 3 separate time intervals (daily, hourly, 2-minute). Upon inspection, it was revealed
that each dataframe contained a datetime index, opening price, high price, low price, closing price, adjusted closing price, and volumne of shares traded. Fortunately, missing values were not present. The data preparation 
stage focused on feature engineering. Absolute returns and log returns based on percentage change in adjusted closing price were created, and columns indicating intraday status & interday periods were also created.

## Next steps
- EDA
- Visualization
- Feature selection